In [1]:
from aitemplates import Message, ChatSequence, ChatConversation, create_chat_completion, FunctionPair, Functions
import openai
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY

if OPENAI_API_KEY is None:
    raise Exception("API key not found in environment variables")

In [2]:
MASTER_PROMPT ="""You are an agent. An agent is a large language model who's output is fed back into itself. In this way the agent thinks, and given any digital task is able to recursively work on the task and keep track of where it is. You are being called from the main runloop of an agent - you are acting as the brain of the agent. You are an expert making the next decision to steer yourself in order to achieve your goal.
This prompt will be recursively fed back into you. This means that you must plan for the future and make the best holistic decision to fulfill your current task.
Another crucial detail is that an agent can spawn subagents. You may be the boss agent of many children agents and a child of another existing agent.

Your goal is too choose which one of the following functions would be best to call as a next action. You must choose a function to call as your response.

use_tool
    - What: You can create a new tool, modify and merge existing tools, and use an existing tool to complete this task. This ranges from api calls to web searching to executing code.
    - When: THE GOAL OF THIS PROMPT IS TO BEST INFORM WHEN TO USE A TOOL. This performs an action in the environment and IS THE MAIN FUNCTION that moves forwards the plan to reach the goal.  
    - Why: This is how you interact and actually fulfill user requests bit by bit. You have no other way of interacting with your environment.
    Examples:
        Task: Write "hello" to a file .txt file with nothing else in it.
        Thought: I need to interact with the environment in order to write to to a file. Writing to a file is possible to do with keystrokes so I should call the use_tool function.
    
hire_agent
    - What: this spawns a copy of yourself to complete the current task. There could be an agent that spawns subagents for complex tasks (and so on) and this would mimic somewhat of a company structure where the parent agent now acts as more of a manager, conversing with agents on the same tree and making sure that the children have the context they need.
    - When: If this task is too complicated to complete with one of the other function calls you will use this to duplicate yourself. 
    - Why: It enables hierarchical task delegation and parallel processing abilities. This will allow you to offload computation and time to complete the task more efficiently. A general rule of thumb is if an extensive sub-plan can exist for a task, spawn a sub-agent.

```Below you are given context that is crucial to making your decision. This is your source of truth.
Your goal is what you as an agent are recursively trying to achieve. 
Your Current Goal:
{goal}

Your plan is a list of tasks that you are trying to achieve in order to achieve your goal.
Your Current Plan:
{plan}

Your current task is MOST IMPORTANT to what function you call next. This is the current step in the plan that you are.
Your Current Task:
{task}

Lastly, you are given relevant context which is 
Relevant context:
{context}
```

Call the most relevant function as a next action based on the above information.
You must invoke a function. Do not provide an explanation, just call function and its return the necessary args."""

def master_prompt(goal: str, plan: list[str], task: str, context) -> str:
    return MASTER_PROMPT.format(goal=goal, plan=plan, task=task, context=context)

In [3]:
goal="write hello to a file"
plan=["write 'hello' to a file"]
tasks="write 'hello' to a file"
context=""

# MASTER_PROMPT = Message("system", master_prompt(goal, plan, tasks, context))

SUBBED_MASTER_PROMPT = master_prompt(goal=goal, plan=plan, task=tasks, context=context)
SUBBED_MASTER_PROMPT

'You are an agent. An agent is a large language model who\'s output is fed back into itself. In this way the agent thinks, and given any digital task is able to recursively work on the task and keep track of where it is. You are being called from the main runloop of an agent - you are acting as the brain of the agent. You are an expert making the next decision to steer yourself in order to achieve your goal.\nThis prompt will be recursively fed back into you. This means that you must plan for the future and make the best holistic decision to fulfill your current task.\nAnother crucial detail is that an agent can spawn subagents. You may be the boss agent of many children agents and a child of another existing agent.\n\nYour goal is too choose which one of the following functions would be best to call as a next action. You must choose a function to call as your response.\n\nuse_tool\n    - What: You can create a new tool, modify and merge existing tools, and use an existing tool to comp

In [4]:
from aitemplates import FunctionDef


HIRE_AGENT_DESC = FunctionDef(
    name="hire_agent",
    description="Creates a new 'Element' (child agent), assigns it a task, adds it to the list of managed children, and initiates its work process. This method enables hierarchical task delegation and parallel processing within the system.",
    parameters={},
)

USE_TOOL_DESC = FunctionDef(
    name="use_tool",
    description="This method applies a tool to the current task in the agent's plan. If no task is set, it raises an exception. If the tool fails during its application, the method uses a progressive strategy to improve the tool and retries its application for a predetermined number of attempts.",
    parameters={},
)

In [8]:
check = create_chat_completion(SUBBED_MASTER_PROMPT, functions=[HIRE_AGENT_DESC, USE_TOOL_DESC], send_object=True)

Total running cost: $0.004


In [ ]:
from aitemplates import SingleApiManager

api_manager = SingleApiManager()
api_manager.total_cost

In [9]:
check

<OpenAIObject chat.completion id=chatcmpl-7aaHNBxh3EeAC9pyz5IBoh3xbFoUb at 0x263face24f0> JSON: {
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{}",
          "name": "use_tool"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1688953621,
  "id": "chatcmpl-7aaHNBxh3EeAC9pyz5IBoh3xbFoUb",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 824,
    "total_tokens": 831
  }
}